In [11]:
!pip install langchain-core langchain-community pymupdf chromadb langchain_google_genai

In [1]:
from langchain_core.documents import Document

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader,DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "pdf-files",
    glob="**/*.pdf", ## Pattern to match files
    loader_cls= PyMuPDFLoader, ##loader class to use
)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'doPDF Ver 8.3 Build 931', 'creator': '', 'creationdate': '2015-11-01T10:54:39+05:45', 'source': 'pdf-files\\constitution.pdf', 'file_path': 'pdf-files\\constitution.pdf', 'total_pages': 240, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20151101105439+05'45'", 'page': 0}, page_content='1 \n \n \n \n \n \nTHE CONSTITUTION OF NEPAL'),
 Document(metadata={'producer': 'doPDF Ver 8.3 Build 931', 'creator': '', 'creationdate': '2015-11-01T10:54:39+05:45', 'source': 'pdf-files\\constitution.pdf', 'file_path': 'pdf-files\\constitution.pdf', 'total_pages': 240, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20151101105439+05'45'", 'page': 1}, page_content='2 \n \nTable of Contents \n \nPreamble   \nPart-1  \nPreliminary  \nPart-2  \nCitizenship  \nPart-3  \nFundamental Rig

In [4]:
import os
import numpy as np
from typing import List, Any
import pickle

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


class ChromaVectorStore:
    def __init__(self, persist_dir: str = "chroma_store",
                 embedding_model_name: str = "all-MiniLM-L6-v2",
                 chunk_size: int = 1000, chunk_overlap: int = 200):
        self.persist_dir = persist_dir
        os.makedirs(self.persist_dir, exist_ok=True)
        self.embedding_function = SentenceTransformerEmbeddings(model_name=embedding_model_name)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vectorstore = None


        # Try to load existing ChromaDB or initialize an empty one
        if os.path.exists(os.path.join(self.persist_dir, "index")):
            print(f"Loading existing ChromaDB from {self.persist_dir}")
            self.vectorstore = Chroma(persist_directory=self.persist_dir,
                                      embedding_function=self.embedding_function)
        else:
            print(f"Initializing new ChromaDB in {self.persist_dir}")
            pass

        print(f"Loaded embedding model: {embedding_model_name}")

    def build_from_documents(self, documents: List[Any]):
        print(f"Building vector store from {len(documents)} raw documents...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
        chunks = text_splitter.split_documents(documents)

        self.vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=self.embedding_function,
            persist_directory=self.persist_dir
        )
        self.vectorstore.persist()
        print(f"Vector store built and saved to {self.persist_dir}")

    def query(self, query_text: str, top_k: int = 5):
        print(f"Querying vector store for: '{query_text}'")
        if not self.vectorstore:
            print("[ERROR] Vector store not initialized. Please build it from documents first.")
            return []
        results = self.vectorstore.similarity_search(query_text, k=top_k)
        return results

In [5]:
# Initialize and build the ChromaDB vector store
chroma_store = ChromaVectorStore()
chroma_store.build_from_documents(pdf_documents)

# Perform a sample query
query_results = chroma_store.query("What is attention mechanism?", top_k=2)

# Display results
for i, doc in enumerate(query_results):
    print(f"\nResult {i+1}:")
    print(f"Page Content: {doc.page_content[:200]}...") # Display first 200 characters
    print(f"Metadata: {doc.metadata}")

C:\Users\USER\AppData\Local\Temp\ipykernel_12756\4187599040.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding_function = SentenceTransformerEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

g:\anaconda-mms\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initializing new ChromaDB in chroma_store
Loaded embedding model: all-MiniLM-L6-v2
Building vector store from 240 raw documents...
Vector store built and saved to chroma_store
Querying vector store for: 'What is attention mechanism?'

Result 1:
Page Content: rights, 
(c) 
in the event of necessity to take action immediately on receipt of 
information that the human rights of any person are being 
violated, to enter any government office or any other place...
Metadata: {'title': '', 'file_path': 'pdf-files\\constitution.pdf', 'page': 166, 'modDate': '', 'producer': 'doPDF Ver 8.3 Build 931', 'format': 'PDF 1.5', 'source': 'pdf-files\\constitution.pdf', 'trapped': '', 'creationdate': '2015-11-01T10:54:39+05:45', 'creationDate': "D:20151101105439+05'45'", 'keywords': '', 'subject': '', 'creator': '', 'author': '', 'moddate': '', 'total_pages': 240}

Result 2:
Page Content: National Human Rights Commission shall perform the following functions: 
(a) 
to inquire, on its own initiative or on

C:\Users\USER\AppData\Local\Temp\ipykernel_12756\4187599040.py:44: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.vectorstore.persist()


In [9]:
from langchain_ollama import ChatOllama


class RAGSearch:
    def __init__(self, persist_dir: str = "chroma_store",
                 embedding_model: str = "all-MiniLM-L6-v2",
                 llm_model: str = "gemini-2.5-flash"):
        self.vectorstore = ChromaVectorStore(persist_dir, embedding_model)
        if self.vectorstore.vectorstore is None or self.vectorstore.vectorstore._collection.count() == 0:
            self.vectorstore.build_from_documents(pdf_documents)
        else:
            print(
                f"Using existing ChromaDB from {self.vectorstore.persist_dir}")

        self.llm = ChatOllama(model="llama3.2:latest")
        print(f" ChatOllama LLM initialized: {llm_model}")

        self.rag_chain = (
            {"context": self.retriever | self.format_docs, "question": 
            RunnablePassthrough[Any]()}
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

        def _format_docs(self, docs: List[Document]) -> str:
            return "\n\n".join([doc.page_content for doc in docs])
        
        def ask(self, query: str):
            return self.rag_chain.invoke(query)

    def search_and_summarize(self, query: str, top_k: int = 5) -> str:
        results = self.vectorstore.query(query, top_k=top_k)
        texts = [doc.page_content for doc in results]
        context = "\n\n".join(texts)
        if not context:
            return "No relevant documents found."
        prompt = f"""Summarize the following context for the query:
        '{query}'\n\nContext:\n{context}\n\nSummary:"""
        response = self.llm.invoke([prompt])
        return response

In [10]:

rag_search = RAGSearch()
query = "Tell me about Nepal cyber security law."
summary = rag_search.search_and_summarize(query, top_k=3)
print("Summary:", summary)

Initializing new ChromaDB in chroma_store
Loaded embedding model: all-MiniLM-L6-v2
Building vector store from 240 raw documents...
Vector store built and saved to chroma_store
 ChatOllama LLM initialized: gemini-2.5-flash


AttributeError: 'RAGSearch' object has no attribute 'retriever'